In [1]:
import cudf
import cugraph
from cugraph import Graph
from numba import cuda, jit, int32
import numpy as np 
import cupy as cp
from math import ceil, floor, log2
import seaborn as sns
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from cugraph.generators import multi_rmat, rmat


GPU_MEM_LIMIT = (18*1024**3) / 1e9


In [2]:
#CUDA Kernels to compute:
# - Degree Distribution
# - Adjacency Lists ( directed and undirected )
# - Edges between neighbors (directed and undirected )

@cuda.jit
def compute_degree_distribution(n, count, percentage):
    """
    - n                 number of vertices
    - count             input cudf series
    - percentage        output cudf series
    """
    i = cuda.threadIdx.x + (cuda.blockIdx.x * cuda.blockDim.x)
    
    if i < count.size:
        percentage[i] = (count[i] / n) * 100
        
@cuda.jit
def undirected_adj_list(nodes, src, dst, out):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    dx = cuda.blockDim.x
    tid = dx * bx + tx
    pos = 0
    if tid < len(nodes):
        for j in range(len(src)):
            if nodes[tid] == src[j]:
                out[tid, pos] = dst[j]
                pos += 1
            elif nodes[tid] == dst[j]:
                out[tid, pos] = src[j]
                pos += 1   

@cuda.jit
def directed_adj_list(nodes, src, dst, out):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    dx = cuda.blockDim.x
    tid = dx * bx + tx
    pos = 0
    if tid < len(nodes):
        node = nodes[tid]
        for j in range(len(src)):
            if node == src[j]:
                out[tid, pos] = dst[j]
                pos += 1

@cuda.jit
def reciprocal_count(A, nodes, src, dst, M, N, out):
    ty = cuda.threadIdx.y; tx = cuda.threadIdx.x
    by = cuda.blockIdx.y; bx = cuda.blockIdx.x
    dy = cuda.blockDim.y; dx = cuda.blockDim.x
    row = dy * by + ty
    column = dx * bx + tx
    
    if row < M and column < N and A[row, column] != -1:
        ngbr = A[row, column]
        node = nodes[row]
        for j in range(len(src)):
            if ngbr == src[j] and node == dst[j]:
                cuda.atomic.add(out, row, 1)
                  
@cuda.jit
def undirected_ngbr_edges(A, src, dst, M, N, out):
    ty = cuda.threadIdx.y; tx = cuda.threadIdx.x
    by = cuda.blockIdx.y; bx = cuda.blockIdx.x
    dy = cuda.blockDim.y; dx = cuda.blockDim.x
    row = dy * by + ty
    column = dx * bx + tx

    if row < M and column < N:
        ngbr = A[row, column]
        for j in range(N):
            second_ngbr = A[row, j]
            if ngbr != -1 and second_ngbr != -1 and ngbr != second_ngbr:
                for k in range(len(src)):
                    if ngbr == src[k] and second_ngbr == dst[k]:
                        cuda.atomic.add(out, row, 1)
                    elif ngbr == dst[k] and second_ngbr == src[k]:
                        cuda.atomic.add(out, row, 1)

@cuda.jit
def directed_ngbr_edges(A, src, dst, M, N, out):
    ty = cuda.threadIdx.y; tx = cuda.threadIdx.x
    by = cuda.blockIdx.y; bx = cuda.blockIdx.x
    dy = cuda.blockDim.y; dx = cuda.blockDim.x
    row = dy * by + ty
    column = dx * bx + tx
    
    if row < M and column < N and A[row, column] != -1:
        ngbr = A[row, column]
        j = 0
        while j < N:
            second_ngbr = A[row, j]
            if A[row, j] != -1 and ngbr != second_ngbr:
                k = 0
                while k < len(src):
                    if ngbr == src[k] and second_ngbr == dst[k]:
                        cuda.atomic.add(out, row, 1)
                    k += 1
            j += 1

@cuda.jit
def lcc_directed(nodes, edges, df_degree, recip, lcc_array):
    tid = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if tid < len(nodes):
        lcc = 0.0
        item = edges[tid]
        if item > 0:
            node_deg = df_degree[tid]
            lcc = item / (node_deg * (node_deg - 1) - (2 * recip[tid]))
        cuda.atomic.add(lcc_array, 0, lcc)

@cuda.jit
def lcc_undirected(nodes, edges, df_degree, lcc_array):
    tid = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if tid < len(nodes):
        lcc = 0.0
        item = edges[tid]
        if item > 0:
            node_deg = df_degree[tid]
            lcc = item / (node_deg * (node_deg - 1))
        cuda.atomic.add(lcc_array, 0, lcc)

@cuda.jit
def gnp_erdos_renyi(p, rng_states, M, N, matrix):
    tid = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    pos = 0
    if tid < M:
        for _ in range(N):
            rnd = xoroshiro128p_uniform_float32(rng_states, tid)
            if rnd <= p:
                matrix[tid, pos] = 1
            pos += 1
                
@cuda.jit
def gnp_erdos_renyi_v2(p, rng_states, M, N, matrix):
    ty = cuda.threadIdx.y; tx = cuda.threadIdx.x
    by = cuda.blockIdx.y; bx = cuda.blockIdx.x
    dy = cuda.blockDim.y; dx = cuda.blockDim.x
    row = dy * by + ty
    column = dx * bx + tx

    if row < M and column < N:
        rnd = xoroshiro128p_uniform_float32(rng_states, row)
        if rnd < p:
            matrix[row, column] = 1
                

In [3]:
def compute_bounds(x, y, bytess) -> int:
    size = ((x * y * bytess) / 1e9)
    if size > GPU_MEM_LIMIT:
        x  = compute_bounds(int(x/2), y, bytess)
    return x

def compute_shortest_path_length(graph, nodes) -> float:
    """
    FIX ME

    - graph     graph object
    - nodes     cupy array containing all graph's nodes (can be range(number_of_nodes))
    """

    distance = 0
    for node in nodes:
        sssp_df = cugraph.filter_unreachable(cugraph.sssp(graph, node))
        distance = distance + sssp_df['distance'].sum()
#    for node in nodes:
#        print(node)
#        ssp_df = cugraph.filter_unreachable(cugraph.shortest_path_length(graph, node))
#        distance += ssp_df['distance'].sum()
#        print(distance)
    return distance / len(nodes)

def degree_distribution_wrapper(n, df) -> cudf.DataFrame:
    """
    - df                cudf dataframe containing in/out/total degree per each node
    - n                 number of vertices    
    - mode              tot OR in OR out degree to specify nothing(??????????????)      
    """
    degree_series = df['degree'].value_counts()
    df_distribution = cudf.DataFrame({'degree': degree_series.index.to_cupy(), \
                                      'count': degree_series.to_cupy(), 'percentage': 0.0})
    size = len(df_distribution)
    compute_degree_distribution.forall(size)(n, df_distribution['count'], df_distribution['percentage'])

    return df_distribution.astype({'percentage': 'float32'})

@cuda.jit
def align(src, const):
    tid = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    if tid < len(src):
        src[tid] = src[tid] + const

In [4]:
def init_cc(n, batch_size, iteration, mod, N, nodes_cp=None, mode='local'):
    """
    - n                 number of vertices of the graph
    - batch_size        range of nodes examined each epoch
    - iteration         current epoch
    - mod               the margin of n / batch_size
    """

    if (batch_size * iteration) <= n:
        start = 0 + (batch_size*(iteration - 1))
        stop = start + batch_size
        M = batch_size
    else:
        start = 0 + (batch_size*(iteration - 1))
        stop = start + mod
        M = mod
    print(start, stop)
    return start, stop, M
        
    
def avg_clustering_coefficient(n, src, dst, df_degree, N, nodes_cp=None, is_directed=False) -> float:
    local_ccs = cp.zeros((1,), dtype='float32')
    M = compute_bounds(n, N, cp.dtype(cp.int32).itemsize)
    epochs = ceil(n / M)
    leftovers = n % M
    epoch = 1
     
    for i in range(1, epochs+1):  
        start, stop, M = init_cc(n, M, i, leftovers, N)
        nodes = cp.arange(start, stop, 1)
        if nodes_cp is not None: nodes = nodes_cp[start : stop]
        matrix = cp.empty((M, N), dtype='int32')
        matrix.fill(-1)
        edgespernode = cp.zeros(M, dtype='int32')
     
        threadsperblock = 1024
        blockspergrid = (M + (threadsperblock -1)) // threadsperblock
        threadsperblock_2D = (32, 32)
        blockspergrid_x = (N + (threadsperblock_2D[1] - 1)) // threadsperblock_2D[1]
        blockspergrid_y = (M + (threadsperblock_2D[0] - 1)) // threadsperblock_2D[0]
        blockspergrid_2D = (blockspergrid_x, blockspergrid_y)

        if is_directed:
            reciprocal = cp.zeros(M, dtype='int32')
            directed_adj_list[blockspergrid, threadsperblock](nodes, src, dst, matrix)
            reciprocal_count[blockspergrid_2D, threadsperblock_2D](matrix, nodes, src, dst, M, N, reciprocal)
            directed_ngbr_edges[blockspergrid_2D, threadsperblock_2D](matrix, src, dst, M, N, edgespernode)       
            lcc_directed[blockspergrid, threadsperblock](nodes, edgespernode, df_degree, reciprocal, local_ccs)
            cuda.synchronize()

        else:
            undirected_adj_list[blockspergrid, threadsperblock](nodes, src, dst, matrix)
            undirected_ngbr_edges[blockspergrid_2D, threadsperblock_2D](matrix, src, dst, M, N, edgespernode)
            lcc_undirected[blockspergrid, threadsperblock](nodes, edgespernode, df_degree['degree'], local_ccs)
            cuda.synchronize()

    return local_ccs[0].get() / n


def random_graph_generator(n, edges) -> cudf.DataFrame:
    L = Graph(directed=True)
    df = cudf.DataFrame({'src': None, 'dst': None})
    p = edges / (n * (n - 1))
    N = n
    M = compute_bounds(n, N, cp.dtype(cp.int32).itemsize)
    np_nodes = np.arange(0, n, 1, dtype='int32')
    epochs = ceil(n / M)
    leftovers = n % M
    epoch = 1

    while epoch <= epochs:
        L.clear()
        start, stop, M = init_cc(n, M, epoch, leftovers, N, mode='rndm')
        matrix = cp.zeros((M, N), dtype='int32')
        threadsperblock = 1024
        blockspergrid = (matrix.shape[0] + (threadsperblock - 1)) // threadsperblock
        rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=42)
        gnp_erdos_renyi[blockspergrid, threadsperblock](p, rng_states, M, N, matrix)
        a_matrix = np.empty((M,N), dtype='int32')
        cp.asnumpy(matrix, stream=None, out=a_matrix)
#        arrays.append(a_matrix)
        L.from_numpy_array(a_matrix)
        df_l = L.view_edge_list()
        df_l.pop('weights')
        size = len(df['src'])
        align.forall(size)(df_l['src'], start)
        df = cudf.concat([df, df_l], ignore_index=True)
        del a_matrix
        del matrix
        epoch += 1
    
#    A = np.concatenate(arrays)
    df.dropna(inplace=True)
    return df

def _multi_rmat (n, edges):
    p = edges / (n * (n-1))
    scale_low = floor(log2(n))
    scale_up = ceil(log2(n))
    print(scale_low, scale_up, int(log2(n)))
#    df = multi_rmat(n_edgelists=1, min_scale=floor(log2(n-1)), max_scale=ceil(log2(n-1)), edge_factor=int(p*(n**2)),
#                 size_distribution=0, edge_distribution=1, seed=1, clip_and_flip=False, scramble_vertex_ids=False)
    df = rmat(scale_low, edges, 0.25, 0.25, 0.5,seed=42, clip_and_flip=False, 
              scramble_vertex_ids=False)    
    return df

In [5]:
#DATA LOADING
edgesfile = '../graphs/ethereum/2020-01-01_2020-01-01/network.csv'
#addressfile = '../graphs/ethereum/2020-01-01_2020-01-01/vertices.csv'
#edgesfile = '../graphs/bitcoin/2016-01-01_2016-01-01/network.csv'
edges_list = cudf.read_csv(edgesfile, delimiter=',', names=['src','dst','wt'], dtype=['int32','int32','float64']) \
                 .drop_duplicates(subset=['src', 'dst'], keep='first') \
                 .dropna(axis=0, how='any')

In [6]:
#REAL GRAPH/NETWORK
G = Graph(directed=True)
G.from_cudf_edgelist(edges_list, source='src', destination='dst', edge_attr='wt',
                     renumber=False, store_transposed=True)

#Computing graph properties
vertex_number = G.number_of_nodes()
edges_number = G.number_of_edges()
tot_degrees = G.degree()            #in and out degrees of all vertices
#tot_degrees = tot_degrees.sort_values(by='vertex', ascending=True, ignore_index=False)
in_degrees = G.in_degree()
out_degrees = G.out_degree()

In [ ]:
g = tot_degrees.to_numpy()

In [ ]:
df_tot = degree_distribution_wrapper(vertex_number, tot_degrees)
df_in = degree_distribution_wrapper(vertex_number, in_degrees)
df_out = degree_distribution_wrapper(vertex_number, out_degrees)

#compute_shortest_path_length(G, G.nodes().to_cupy())


In [ ]:
totals = sns.displot(df_tot.to_pandas(), x='degree', kde=True)

In [ ]:
outs = sns.displot(df_in.to_pandas(), x='degree', kde=True)

In [ ]:
ins = sns.displot(df_out.to_pandas(), x='degree', kde=True)

In [7]:
N = tot_degrees['degree'].max()
avg_cc = avg_clustering_coefficient(vertex_number, edges_list['src'], edges_list['dst'], 
                                    tot_degrees['degree'], N, is_directed=False)


0 159295


In [8]:
#Find Main Component
df_components = cugraph.connected_components(G, connection='weak')
target_label = df_components['labels'].mode()[0]
df_nodes = df_components[df_components['labels'] == target_label]
edges_list_mc = edges_list.loc[edges_list['src'].isin(df_nodes['vertex'])]
G_mc = Graph(directed=True)
G_mc.from_cudf_edgelist(edges_list_mc, source='src', destination='dst', edge_attr='wt', renumber=True)

#Computing Main component properties
vertex_number_mc = G_mc.number_of_nodes()
edges_number_mc = G_mc.number_of_edges()
tot_degrees_mc = G_mc.degree().sort_values(by='vertex', ignore_index=True)
in_degrees_mc = G_mc.in_degree().sort_values(by='vertex', ignore_index=True)
out_degrees_mc = G_mc.out_degree().sort_values(by='vertex', ignore_index=True)
vertices_mc = G_mc.nodes().sort_values(ascending=True).to_cupy()

In [ ]:
df = cugraph.filter_unreachable(cugraph.shortest_path_length(G_mc, 1).sort_values(by='vertex'))

In [ ]:
df_tot_mc = degree_distribution_wrapper(vertex_number_mc, tot_degrees_mc, mode='tot')
df_in_mc = degree_distribution_wrapper(vertex_number_mc, in_degrees_mc, mode='in')
df_out_mc = degree_distribution_wrapper(vertex_number_mc, out_degrees_mc, mode='out')

#compute_shortest_path_length(G_mc, G_mc.nodes().to_cupy())

In [ ]:
tot_mc = sns.displot(df_tot_mc.to_pandas(), x='degree', kde=True)

In [ ]:
in_mc = sns.displot(df_in_mc.to_pandas(), x='degree', kde=True)

In [ ]:
out_mc = sns.displot(df_out_mc.to_pandas(), x='degree', kde=True)

In [9]:
N = tot_degrees['degree'].max()
mc_avg_cc = avg_clustering_coefficient(vertex_number_mc, edges_list_mc['src'], 
                                       edges_list_mc['dst'], tot_degrees_mc['degree'], 
                                       N, nodes_cp=vertices_mc, is_directed=True)


0 144335


In [ ]:
edge_list_rnd = random_graph_generator(vertex_number, edges_number)

G_rnd = Graph(directed=True)
G_rnd.from_cudf_edgelist(edge_list_rnd, source='src', destination='dst', renumber=False)


vertex_number_rnd = G_rnd.number_of_nodes()
edges_number_rnd = G_rnd.number_of_edges()
tot_degrees_rnd = G_rnd.degree()
in_degrees_rnd = G_rnd.in_degree()
out_degrees_rnd = G_rnd.out_degree()
#edge_list_rnd = G_rnd.view_edge_list()

In [ ]:
edge_list_rnd

In [ ]:
df_tot_rnd = degree_distribution_wrapper(vertex_number_rnd, tot_degrees_rnd, mode='tot')
df_in_rnd = degree_distribution_wrapper( vertex_number_rnd, in_degrees_rnd, mode='in')
df_out_rnd = degree_distribution_wrapper(vertex_number_rnd, out_degrees_rnd, mode='out')

In [ ]:
tot_rnd = sns.displot(df_tot_rnd.to_pandas(), x='degree', kde=True)

In [ ]:
out_rnd = sns.displot(df_out_rnd.to_pandas(), x='degree', kde=True)

In [ ]:
in_rnd = sns.displot(df_in_rnd.to_pandas(), x='degree', kde=True)

In [ ]:
N = tot_degrees_rnd['degree'].max()
avg_cc_rnd = avg_clustering_coefficient(vertex_number_rnd, edge_list_rnd['src'], 
                                        edge_list_rnd['dst'], tot_degrees_rnd['degree'],
                                        N, is_directed=True) 


In [ ]:
df_components = cugraph.connected_components(G_rnd, connection='weak')
target_label = df_components['labels'].mode()[0]
df_nodes = df_components[df_components['labels'] == target_label]
edges_list_rnd_mc = edge_list_rnd.loc[edge_list_rnd['src'].isin(df_nodes['vertex'])]
G_rnd_mc = Graph(directed=True)
G_rnd_mc.from_cudf_edgelist(edges_list_rnd_mc, source='src', destination='dst', renumber=True)

In [ ]:
vertex_number_rnd_mc = G_rnd_mc.number_of_nodes()
edges_number_rnd_mc = G_rnd_mc.number_of_edges()

tot_degrees_rnd_mc = G_rnd_mc.degree().sort_values(by='vertex', ignore_index=True)
in_degrees_rnd_mc = G_rnd_mc.in_degree().sort_values(by='vertex', ignore_index=True)
out_degrees_rnd_mc = G_rnd_mc.out_degree().sort_values(by='vertex', ignore_index=True)
vertices_rnd_mc = G_rnd_mc.nodes().sort_values(ascending=True).to_cupy()

In [ ]:
N = tot_degrees_rnd_mc['degree'].max()
avg_cc_rnd_mc = avg_clustering_coefficient(vertex_number_rnd_mc, edges_list_rnd_mc['src'],
                                           edges_list_rnd_mc['dst'], tot_degrees_rnd_mc['degree'],
                                           N, nodes_cp=vertices_rnd_mc, is_directed=True)

In [ ]:
df = cugraph.shortest_path_length(G_mc, 0, 1)

In [ ]:
df

In [ ]:
df_1 = cugraph.filter_unreachable(cugraph.shortest_path_length(G_mc, 0))

In [ ]:
df_1.sort_values(by='vertex')
df_1['distance'].sum()

In [ ]:
compute_shortest_path_length(G_mc, vertices_mc)

In [ ]:
f.sort_values(by='vertex')

In [ ]:
f